In [1]:
import re
from typing import final
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

from dotenv import load_dotenv

load_dotenv('./env/.env')

llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

splitter = RecursiveCharacterTextSplitter(
    separators="\n",
    chunk_size = 600,
    chunk_overlap = 100
)

loader = UnstructuredFileLoader("./files/chapter_one.docx")
docs = loader.load_and_split(text_splitter=splitter)

# 캐시에 embeddings가 있는 지 확인한후, 없으면 embeddings를 캐시에 저장
embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = Chroma.from_documents(docs, cached_embeddings)

# Retriever는 한개의 string을 입력받고 document들의 list를 출력한다.
retriever = vectorstore.as_retriever()

map_doc_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Use the following portion of a long document to see if any of the text is relevant to answer the question. Return any relevant text verbatim. If there is no relevant text, return : ''
            -------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

map_doc_chain = map_doc_prompt | llm


def map_docs(inputs):
    documents = inputs["documents"]
    question = inputs["question"]
    return "\n\n".join( 
        map_doc_chain.invoke(
            {"context": doc.page_content, "question": question}
        ).content
        for doc in documents
        # for문을 통해 document들을 순회하며, document의 page_content와 사용자 question을 입력받아 작업을 실행한다
    )

# invoke를 통해 arg가 map_chain에 전달되고, retriever를 통해 많은 document가 출력된다
# RunnableLambda는 chain과 그 내부 어디에서든 function을 호출할 수 있게 해준다.
map_chain = {
    "documents": retriever,
    "question": RunnablePassthrough(),
} | RunnableLambda(map_docs)

# llm에 전달될 final_prompt를 생성한다.
# 여기서 context는 llm이 추출한 여러 document들의 작은 부분들을 모아서 만들어진다.
final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Use the following portion of a long document to see if any of the text is relevant to answer the question. 
            Return any relevant text verbatim. 
            If there is no relevant text, return : ''
            -------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

# RunnablePassthrough는 입력값(Description Victory Mansions)을 그대로 출력함(해당 값이 question에 그대로 작성됨)
chain = {"context": map_chain, "question": RunnablePassthrough()} | final_prompt | llm

chain.invoke("How many ministries are mentioned")




AIMessage(content='The text mentions four ministries: the Ministry of Truth, the Ministry of Peace, the Ministry of Love, and the Ministry of Plenty.')